In [ ]:
#necessary libs
import numpy as np
import pandas as pd
import os
from tqdm import tqdm, trange
#multiprocessing to speed up things
from multiprocessing import Pool
#Image Processing Libs
from IPython.display import display, Image
import cv2 as cv

In [ ]:
#for inspecting data
PATH = '../input/celebrities-100k/100k/100k/'
Image(PATH + '168202.jpg')

# MODEL

In [ ]:
#keras imports
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Input, Flatten, Reshape, Conv2DTranspose 
from tensorflow.keras.layers import Embedding, BatchNormalization, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence, plot_model
from tensorflow.python.keras import backend
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

In [ ]:
BATCH_SIZE = 128
NUM_EPOCHS = 125
LR = 0.001
NUM_SAMPLES = len(os.listdir(PATH))
NUM_SAMPLES

In [ ]:
model = Sequential()

model.add(Embedding(NUM_SAMPLES, 32, input_length=1))
model.add(Flatten(name='pre_encoder'))
model.add(Reshape((32, 1, 1), name='encoder'))

model.add(Conv2DTranspose(128, 4, strides=(1, 2), padding="same"))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Conv2DTranspose(256, 4, strides=(1, 2), padding="same"))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Conv2DTranspose(512, 4, strides=(1, 2), padding="same"))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Conv2DTranspose(256, 4, strides=(1, 2), padding="same"))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Conv2DTranspose(128, 4, strides=(1, 2), padding="same"))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Conv2DTranspose(128, 4, strides=(2, 2), padding="same"))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Conv2DTranspose(3, 4, strides=2, padding="same", activation="sigmoid"))

model.compile(optimizer=Adam(learning_rate=LR), loss='mse')

In [ ]:
model.summary()

In [ ]:
plot_model(model, to_file='model.png', show_shapes=True)
display(Image('model.png'))

# PREPROCESSING + DATA GENERATION

In [ ]:
X = np.expand_dims(np.arange(NUM_SAMPLES), axis=1)
X.shape

In [ ]:
def save_image(x, fname):
    img = x * 255.0
    img = cv.cvtColor(img.astype(np.float32), cv.COLOR_RGB2BGR)
    cv.imwrite(fname, img)

In [ ]:
# Keras Data Generator Class, can't fit 100k pics in memory at once :(
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, n_samples=NUM_SAMPLES, batch_size=BATCH_SIZE, dim=(128, 128), n_channels=3, 
                 path=PATH, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = np.arange(n_samples)
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.path = path
        self.imgs = os.listdir(PATH)
        self.currepoch = 0
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
        # save images so we can check progress over epochs
        y_faces = model.predict(X[:8], batch_size=8)
        save_image(cv.hconcat(y_faces), f"epoch{self.currepoch:03}.jpg")
        
        self.currepoch += 1

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        Y = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img = cv.imread(PATH + self.imgs[ID])
            img = cv.cvtColor(img, cv.COLOR_BGR2RGB) / 255.0
            Y[i,] = img

        return np.expand_dims(list_IDs_temp, axis=1), Y

# TRAINING

In [ ]:
import tensorflow as tf
print("GPU Available: ", tf.test.is_gpu_available())
print("TF Version: ", tf.__version__)

In [ ]:
datagen = DataGenerator(batch_size=BATCH_SIZE) 

early_stopping = EarlyStopping(
    monitor='loss', 
    patience=5
)

history = model.fit(datagen, epochs=NUM_EPOCHS, callbacks=[early_stopping], verbose=2)

In [ ]:
model.save('model.h5')

### Encoder Model

In [ ]:
encoder = Model(inputs=model.input,
                outputs=model.get_layer('pre_encoder').output)

In [ ]:
x_enc = encoder.predict(X, batch_size=8)

In [ ]:
x_mean = np.mean(x_enc, axis=0)
x_stds = np.std(x_enc, axis=0)
x_cov = np.cov((x_enc - x_mean).T)
e, v = np.linalg.eig(x_cov)

In [ ]:
from tensorflow.python.keras import backend

tf.compat.v1.disable_eager_execution() #this will allow us to get to intermediate layers

#this is a really ugly work around
#I want the performance gains from eager execution but cant use function with it
#so make sure this is run in order
loaded = load_model('model.h5') #built without eager execution

## TODO MAKE A DECODER that always works
try:
    decoder = backend.function([loaded.get_layer('encoder').input, backend.symbolic_learning_phase()],
                      [loaded.layers[-1].output])
except:
    print("decoder failed to be built.")

In [ ]:
rand_vecs = np.random.normal(0.0, 1.0, (8, 32))

In [ ]:
x_vecs = x_mean + np.dot(v, (rand_vecs * (e ** 0.5).T).T
try:
    y_faces = decoder([x_vecs, 0])[0]  
except:    
    y_faces = np.array([])

In [ ]:
for i in range(y_faces.shape[0]):
    save_image(y_faces[i], f'rand{i}.jpg')
    display(Image(f'rand{i}.jpg'))

In [ ]:
import json

#to use with js
with open("means.json", 'w') as f:
    json.dump(x_mean.tolist(), f)

with open("stds.json", 'w') as f:
    json.dump(x_stds.tolist(), f)

with open("evals.json", 'w') as f:
    json.dump(e.tolist(), f)

with open("evecs.json", 'w') as f:
    json.dump(v.tolist(), f)
    
# np.save('means.npy', x_mean)
# np.save('stds.npy', x_stds)
# np.save('evals.npy', e)
# np.save('evecs.npy', v)

In [ ]:
!pip install tensorflowjs --user

In [ ]:
#convert to tf.js format so we can use w/ web app
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "FaceModel")

In [ ]:
images = []
try:
    for i in range(1, NUM_EPOCHS, 5):
        images.append(cv.imread(f"epoch{i:03}.jpg"))
    save_image(cv.v_concat(images), "epochs.jpg")
    display(Image("epochs.jpg"))
except:
    print("Collage image failed ;/")